In [1]:
import tensorflow as tf
from tensorflow import keras
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from imutils import paths
import random
import cv2
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense, Dropout, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix, classification_report
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import flwr as fl
# Load and compile Keras model


In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)


In [3]:
BATCH_SIZE = 8
train_data_dir = 'D:/TQ/Balanced_Another/Client4/train/'

test_data_dir = 'D:/TQ/Balanced_Another/Client4/val/'


In [4]:

train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # Set the validation split percentage
)
test_datagen = ImageDataGenerator(rescale=1./255,
                                  shear_range=0.2,
                                  zoom_range=0.2,
                                  horizontal_flip=True)




In [5]:
print("Class Counts in Training Samples:")
total_classes = sorted(os.listdir(train_data_dir))
print(total_classes)
for category in total_classes:
    category_path = os.path.join(train_data_dir, category)
    sample_count = len(os.listdir(category_path))
    print(f"{category}: {sample_count} samples")


Class Counts in Training Samples:
['Animal_Abuse', 'Arson', 'Fight', 'Normal', 'Riot', 'Traffic_Accident', 'Train_Accident']
Animal_Abuse: 124 samples
Arson: 136 samples
Fight: 342 samples
Normal: 199 samples
Riot: 0 samples
Traffic_Accident: 179 samples
Train_Accident: 150 samples


In [6]:
train_classes = sorted(os.listdir(train_data_dir))
test_classes = sorted(os.listdir(test_data_dir))


In [34]:
base_model = tf.keras.applications.MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
# Freeze the base model layers
base_model.trainable = False

# Add custom classification layers on top of the base model
inputs = Input(shape=(224, 224, 3))
x = base_model(inputs, training=False)
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)  # Add dropout layer with a dropout rate of 0.5
outputs = Dense(7, activation='softmax')(x)  # Assuming binary classification

# Create the full model
model4 = Model(inputs, outputs)
optimizer = Adam(learning_rate=0.001)
model4.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy'])


C:\Users\cslen\AppData\Local\Programs\Python\Python310\lib\random.py:370: DeprecationWarning: non-integer arguments to randrange() have been deprecated since Python 3.10 and will be removed in a subsequent version
  return self.randrange(a, b+1)


In [7]:
traingen = train_datagen.flow_from_directory(train_data_dir,
                                                   target_size=(225, 225),
                                                   class_mode='categorical',
                                                   classes=total_classes,
                                                   subset='training',
                                                   batch_size=BATCH_SIZE, 
                                                   shuffle=True,
                                                   seed=42)

validgen = train_datagen.flow_from_directory(train_data_dir,
                                               target_size=(225, 225),
                                               class_mode='categorical',
                                               classes=total_classes,
                                               subset='validation',
                                               batch_size=BATCH_SIZE,
                                               shuffle=True,
                                               seed=42)

testgen = test_datagen.flow_from_directory(test_data_dir,
                                             target_size=(225, 225),
                                             class_mode='categorical',
                                             classes=total_classes,
                                             batch_size=1,
                                             shuffle=False,
                                             seed=42)


Found 907 images belonging to 7 classes.
Found 223 images belonging to 7 classes.
Found 350 images belonging to 7 classes.


In [36]:
early_stopping = EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True)
callbacks = [early_stopping]


In [37]:
history = model4.fit(
            traingen,
            epochs=5,
            steps_per_epoch=len(traingen),
            validation_data=validgen,
            validation_steps=len(validgen),
            callbacks=callbacks,
            verbose = 1
)

Epoch 1/5


C:\Users\cslen\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\preprocessing\image.py:2616: DeprecationWarning: Please import `affine_transform` from the `scipy.ndimage` namespace; the `scipy.ndimage.interpolation` namespace is deprecated and will be removed in SciPy 2.0.0.
  ndimage.interpolation.affine_transform(


114/114 [==============================] - 19s 157ms/step - loss: 0.8812 - accuracy: 0.6979 - val_loss: 1.1373 - val_accuracy: 0.5695
Epoch 2/5
114/114 [==============================] - 16s 143ms/step - loss: 0.2434 - accuracy: 0.9272 - val_loss: 1.0855 - val_accuracy: 0.5830
Epoch 3/5
114/114 [==============================] - 21s 180ms/step - loss: 0.1584 - accuracy: 0.9647 - val_loss: 1.2049 - val_accuracy: 0.5874
Epoch 4/5
114/114 [==============================] - 19s 166ms/step - loss: 0.1337 - accuracy: 0.9537 - val_loss: 1.2941 - val_accuracy: 0.6278
Epoch 5/5
114/114 [==============================] - 20s 176ms/step - loss: 0.0938 - accuracy: 0.9813 - val_loss: 1.5073 - val_accuracy: 0.5605


In [38]:
#model = tf.keras.models.load_model("D:/TQ/Federated/Client4_Centralized.h5")

In [39]:
y_true = testgen.classes

# Calculate predictions on the test data
y_pred = model4.predict(testgen)
y_pred_labels = np.argmax(y_pred, axis=1)  # Get the predicted class labels

350/350 [==============================] - 7s 20ms/step


In [40]:
print("Accuracy:")
test_loss, test_accuracy = model.evaluate(testgen, steps=len(testgen), verbose=0)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")


Accuracy:
Test Loss: 2.114243268966675
Test Accuracy: 0.7485714554786682


In [41]:
# Calculate the confusion matrix
cm2 = confusion_matrix(y_true, y_pred_labels)
print("Confusion Matrix Before Aggregation:")
print(cm2)

Confusion Matrix Before Aggregation:
[[30  0  0  1  0  0  0]
 [ 0 34  0  1  0  0  0]
 [ 0  0 74 12  0  0  0]
 [ 0  0  1 44  0  5  0]
 [ 0  0  3 50  0  1 10]
 [ 0  1  0  2  0 42  0]
 [ 0  0  0  0  0  1 38]]


In [43]:
import pandas as pd
from openpyxl import Workbook

categories = ['Animal_Abuse', 'Arson', 'Fight', 'Normal', 'Riot', 'Traffic_Accident', 'Train_Accident']
df_cm = pd.DataFrame(cm2, index=categories, columns=categories)

# Write the confusion matrix DataFrame to an Excel file
excel_writer = pd.ExcelWriter('D:/TQ/Federated/confusion_matrix.xlsx')
# Create an Excel workbook and write the confusion matrix DataFrame to a sheet
wb = Workbook()
ws = wb.active
ws.title = 'Confusion Matrix'

# Write the column names
ws.append([''] + df_cm.columns.tolist())

# Write the confusion matrix data
for index, row in df_cm.iterrows():
    ws.append([index] + row.tolist())

# Save the workbook to an Excel file
wb.save('D:/TQ/Federated/confusion_matrix4.xlsx')

print("Confusion matrix written to confusion_matrix.xlsx")

Confusion matrix written to confusion_matrix.xlsx


In [44]:
model.save("Client4_Centralized.h5")

In [29]:
from sklearn.metrics import classification_report 
print(classification_report(y_true, y_pred_labels, target_names=total_classes))


                  precision    recall  f1-score   support

    Animal_Abuse       1.00      0.94      0.97        31
           Arson       0.97      0.97      0.97        35
           Fight       0.82      0.85      0.83        86
          Normal       0.45      0.88      0.59        50
            Riot       0.00      0.00      0.00        64
Traffic_Accident       0.78      0.93      0.85        45
  Train_Accident       0.84      0.97      0.90        39

        accuracy                           0.74       350
       macro avg       0.69      0.79      0.73       350
    weighted avg       0.65      0.74      0.68       350



C:\Users\cslen\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\cslen\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\cslen\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

In [8]:
client4 = tf.keras.models.load_model('clinet4_30_rounds.h5')

C:\Users\cslen\AppData\Local\Programs\Python\Python310\lib\random.py:370: DeprecationWarning: non-integer arguments to randrange() have been deprecated since Python 3.10 and will be removed in a subsequent version
  return self.randrange(a, b+1)


In [9]:
y_true = testgen.classes

# Calculate predictions on the test data
y_pred = client4.predict(testgen)
y_pred_labels = np.argmax(y_pred, axis=1)  # Get the predicted class labels
test_loss, test_accuracy = client4.evaluate(testgen, steps=len(testgen), verbose=0)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

print(classification_report(y_true, y_pred_labels, target_names=total_classes))


C:\Users\cslen\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\preprocessing\image.py:2616: DeprecationWarning: Please import `affine_transform` from the `scipy.ndimage` namespace; the `scipy.ndimage.interpolation` namespace is deprecated and will be removed in SciPy 2.0.0.
  ndimage.interpolation.affine_transform(


350/350 [==============================] - 8s 19ms/step
Test Loss: 1.3470957279205322
Test Accuracy: 0.6971428394317627
                  precision    recall  f1-score   support

    Animal_Abuse       1.00      0.97      0.98        31
           Arson       1.00      0.86      0.92        35
           Fight       0.94      0.17      0.29        86
          Normal       0.32      0.92      0.47        50
            Riot       0.98      0.67      0.80        64
Traffic_Accident       0.88      1.00      0.94        45
  Train_Accident       1.00      0.90      0.95        39

        accuracy                           0.70       350
       macro avg       0.87      0.78      0.76       350
    weighted avg       0.87      0.70      0.69       350



In [10]:
cm = confusion_matrix(y_true, y_pred_labels)
print("Confusion Matrix Before Aggregation:")
print(cm)

Confusion Matrix Before Aggregation:
[[30  0  1  0  0  0  0]
 [ 0 30  0  5  0  0  0]
 [ 0  0 15 70  1  0  0]
 [ 0  0  0 46  0  4  0]
 [ 0  0  0 21 43  0  0]
 [ 0  0  0  0  0 45  0]
 [ 0  0  0  2  0  2 35]]


In [33]:
import pandas as pd
from openpyxl import Workbook

categories = ['Animal_Abuse', 'Arson', 'Fight', 'Normal', 'Riot', 'Traffic_Accident', 'Train_Accident']
df_cm = pd.DataFrame(cm, index=categories, columns=categories)

# Write the confusion matrix DataFrame to an Excel file
excel_writer = pd.ExcelWriter('D:/TQ/Federated/confusion_matrix.xlsx')
# Create an Excel workbook and write the confusion matrix DataFrame to a sheet
wb = Workbook()
ws = wb.active
ws.title = 'Confusion Matrix'

# Write the column names
ws.append([''] + df_cm.columns.tolist())

# Write the confusion matrix data
for index, row in df_cm.iterrows():
    ws.append([index] + row.tolist())

# Save the workbook to an Excel file
wb.save('D:/TQ/Federated/confusion_matrix4f.xlsx')

print("Confusion matrix written to confusion_matrix.xlsx")

Confusion matrix written to confusion_matrix.xlsx
